<a href="https://colab.research.google.com/github/nprimavera/Programmable-Embroidery/blob/main/Soccer_ball_with_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import struct
import math

def write_jef_file(filename, commands):
    with open(filename, 'wb') as f:
        f.write(b'JEF\x00\x00\x00\x00')
        for command in commands:
            f.write(command)

def move(x, y):
    return struct.pack('<BBhh', 0x01, 0x01, int(x), int(y))

def color_change(color_index):
    return struct.pack('<BBH', 0x80, 0x04, color_index)

def write_text(text, start_x, start_y):
    text_commands = []
    for char in text:
        char_code = ord(char)
        text_commands.append(color_change(1))  # Switch to a different color for the text
        text_commands.append(move(start_x, start_y))  # Move to the starting position for the character
        text_commands.append(struct.pack('<BB', 0x83, char_code))  # Add character command
        start_x += 10  # Move horizontally for the next character
    return text_commands

def main():
    diameter_cm = 10  # Diameter of the soccer ball patch in cm
    stitch_length = 10  # Length of the stitch in 1/10 mm
    radius = diameter_cm * 0.5

    # Calculate the number of stitches required
    num_circle_points = 40  # Number of points to approximate the circle
    num_hex_points = 6  # Number of points for hexagon pattern
    num_pent_points = 5  # Number of points for pentagon pattern
    commands = []

    # Move to the starting position for the circle
    start_x = radius * math.cos(0)
    start_y = radius * math.sin(0)
    commands.append(move(start_x, start_y))

    # Generate circle points
    for i in range(num_circle_points + 1):
        angle = 2 * math.pi * i / num_circle_points
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        commands.append(move(x, y))

    # Switch color for hexagon pattern
    commands.append(color_change(1))

    # Generate hexagon pattern
    hex_radius = radius * 0.85
    for i in range(num_hex_points + 1):
        angle = 2 * math.pi * i / num_hex_points
        x = hex_radius * math.cos(angle)
        y = hex_radius * math.sin(angle)
        commands.append(move(x, y))

    # Switch color for pentagon pattern
    commands.append(color_change(2))

    # Generate pentagon pattern
    pent_radius = radius * 0.55
    for i in range(num_pent_points + 1):
        angle = 2 * math.pi * i / num_pent_points
        x = pent_radius * math.cos(angle + math.pi / num_pent_points)
        y = pent_radius * math.sin(angle + math.pi / num_pent_points)
        commands.append(move(x, y))

    # Write "AC MILAN" text
    commands += write_text("AC MILAN", -25, 25)  # Adjust the starting position as needed

    # End of design
    commands.append(color_change(0))  # End of design

    # Write commands to JEF file
    write_jef_file('soccer_ball_patch_with_text.jef', commands)

if __name__ == "__main__":
    main()